In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Height"
cohort = "GSE181339"

# Input paths
in_trait_dir = "../../input/GEO/Height"
in_cohort_dir = "../../input/GEO/Height/GSE181339"

# Output paths
out_data_file = "../../output/preprocess/Height/GSE181339.csv"
out_gene_data_file = "../../output/preprocess/Height/gene_data/GSE181339.csv"
out_clinical_data_file = "../../output/preprocess/Height/clinical_data/GSE181339.csv"
json_path = "../../output/preprocess/Height/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# 1. Gene Expression Data Availability
# Based on the series description, this appears to be a gene expression study
# "For the microarray experiment..." suggests gene expression data is available
is_gene_available = True

# 2. Variable Availability and Data Type Conversion

# 2.1 Data Availability
trait_row = 1  # 'group' contains weight status information (NW, OW/OB, MONW)
age_row = 2    # 'age' is available
gender_row = 0  # 'gender' is available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert the group value to binary form (0 for normal weight, 1 for overweight/obese or MONW)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    if value.upper() == "NW":
        return 0  # Normal weight
    elif value.upper() in ["OW/OB", "MONW"]:
        return 1  # Overweight/obese or Metabolically Obese Normal-Weight
    else:
        return None

def convert_age(value):
    """Convert age string to numeric value"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip()
    
    try:
        return float(value)
    except (ValueError, TypeError):
        return None

def convert_gender(value):
    """Convert gender string to binary (0 for female, 1 for male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract the value after the colon if present
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
    
    if value.lower() == "woman" or value.lower() == "female":
        return 0
    elif value.lower() == "man" or value.lower() == "male":
        return 1
    else:
        return None

# 3. Save Metadata
# Since trait_row is not None, trait data is available
is_trait_available = trait_row is not None

# Conduct initial filtering on the usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Since trait_row is not None, we proceed with clinical data extraction
# Create a DataFrame from the sample characteristics dictionary provided in the task
sample_characteristics = {
    0: ['gender: Man', 'gender: Woman'],
    1: ['group: NW', 'group: OW/OB', 'group: MONW'],
    2: ['age: 21', 'age: 23', 'age: 10', 'age: 17', 'age: 11', 'age: 1', 'age: 18', 'age: 12', 'age: 8', 'age: 14', 'age: 26', 'age: 4', 'age: 2', 'age: 3', 'age: 7', 'age: 13', 'age: 15', 'age: 9', 'age: 30', 'age: 19'],
    3: ['fasting time: 6hr', 'fasting time: 4hr'],
    4: ['timepoint: 0months', 'timepoint: 6months']
}

# Convert to a format that geo_select_clinical_features can work with
# Create a dataframe with the sample characteristics
clinical_data = pd.DataFrame.from_dict(sample_characteristics, orient='index')
clinical_data.index.name = 'row_id'
clinical_data = clinical_data.reset_index()

# Extract clinical features
clinical_features = geo_select_clinical_features(
    clinical_df=clinical_data,
    trait=trait,
    trait_row=trait_row,
    convert_trait=convert_trait,
    age_row=age_row,
    convert_age=convert_age,
    gender_row=gender_row,
    convert_gender=convert_gender
)

# Preview the extracted clinical features
preview = preview_df(clinical_features)
print("Preview of clinical features:", preview)

# Save the clinical data
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
clinical_features.to_csv(out_clinical_data_file, index=False)
print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
I'll debug the code to properly handle GEO dataset files and extract clinical features.

```python
import os
import json
import pandas as pd
import glob
from typing import Optional, Callable, Dict, Any

# First, let's explore what files are available in the directory
print(f"Exploring directory: {in_cohort_dir}")
available_files = glob.glob(os.path.join(in_cohort_dir, "*"))
print(f"Available files: {available_files}")

# GEO data is typically stored in _series_matrix.txt files
series_matrix_files = glob.glob(os.path.join(in_cohort_dir, "*_series_matrix.txt*"))
if series_matrix_files:
    matrix_file = series_matrix_files[0]
    print(f"Found series matrix file: {matrix_file}")
    
    # Read the file line by line to extract sample characteristics
    sample_char_dict = {}
    current_line_idx = 0
    with open(matrix_file, 'r') as f:
        for line in f:
            if line.startswith('!Sample_characteristics_ch1'):
                parts = line.strip().split('\t')
                if len(parts) > 1:  # Ensure there's at least one sample
                    # Remove the prefix to get just the values
                    values = [p.replace('!Sample_characteristics_ch1 = ', '') for p in parts]
                    sample_char_dict[current_line_idx] = values
                    current_line_idx += 1
            elif line.startswith('!Sample_title'):
                # Sample titles can sometimes contain useful information
                parts = line.strip().split('\t')
                if len(parts) > 1:
                    values = [p.replace('!Sample_title = ', '') for p in parts]
                    sample_char_dict[current_line_idx] = values
                    current_line_idx += 1
                    
    # If we've collected any sample characteristics, convert to DataFrame
    if sample_char_dict:
        clinical_data = pd.DataFrame(sample_char_dict).T
        # Print a preview of what we found
        print("Sample characteristics preview:")
        for idx, row in clinical_data.iterrows():
            print(f"Row {idx}: {row.unique()[:5]}...")
    else:
        clinical_data = pd.DataFrame()
        print("No sample characteristics found in series matrix file.")
else:
    # If no series matrix file, try to find a soft file
    soft_files = glob.glob(os.path.join(in_cohort_dir, "*.soft*"))
    if soft_files:
        soft_file = soft_files[0]
        print(f"Found SOFT file: {soft_file}")
        
        # Read the SOFT file to extract sample characteristics
        sample_char_dict = {}
        current_line_idx = 0
        with open(soft_file, 'r') as f:
            in_sample_section = False
            current_sample = None
            for line in f:
                if line.startswith('^SAMPLE'):
                    in_sample_section = True
                    current_sample = []
                elif line.startswith('!Sample_characteristics_ch1'):
                    if in_sample_section:
                        current_sample.append(line.strip().split(' = ')[1])
                elif line.startswith('!sample_table_end'):
                    if in_sample_section and current_sample:
                        sample_char_dict[current_line_idx] = current_sample
                        current_line_idx += 1
                        current_sample = None
                        in_sample_section = False
                        
        if sample_char_dict:
            clinical_data = pd.DataFrame(sample_char_dict).T
            print("Sample characteristics preview from SOFT file:")
            for idx, row in clinical_data.iterrows():
                print(f"Row {idx}: {row.unique()[:5]}...")
        else:
            clinical_data = pd.DataFrame()
            print("No sample characteristics found in SOFT file.")
    else:
        # As a last resort, try to find any text files
        txt_files = glob.glob(os.path.join(in_cohort_dir, "*.txt"))
        if txt_files:
            print(f"Found text files but no recognized GEO format: {txt_files}")
            clinical_data = pd.DataFrame()
        else:
            print("No recognizable data files found.")
            clinical_data = pd.DataFrame()

# Analyze what we have and make decisions about data availability
is_gene_available = True  # Assuming gene expression data exists unless determined otherwise

# Determine if height-related data is available in the clinical data
trait_row = None
age_row = None
gender_row = None

if not clinical_data.empty:
    # Check each row for trait, age, and gender data
    for row_idx in range(len(clinical_data)):
        row_values = clinical_data.iloc[row_idx].astype(str)
        row_text = ' '.join(row_values).lower()
        
        # Check for trait (Height)
        if 'height' in row_text and trait_row is None:
            unique_values = row_values.unique()
            if len(unique_values) > 1:  # More than one unique value
                trait_row = row_idx
                print(f"Found trait data (Height) in row {row_idx}: {unique_values[:5]}")
        
        # Check for age
        if ('age' in row_text or 'years' in row_text) and age_row is None:
            unique_values = row_values.unique()
            if len(unique_values) > 1:  # More than one unique value
                age_row = row_idx
                print(f"Found age data in row {row_idx}: {unique_values[:5]}")
        
        # Check for gender
        if ('gender' in row_text or 'sex' in row_text) and gender_row is None:
            unique_values = row_values.unique()
            if len(unique_values) > 1:  # More than one unique value
                gender_row = row_idx
                print(f"Found gender data in row {row_idx}: {unique_values[:5]}")

# Define conversion functions based on the identified data structure
def convert_trait(value):
    """Convert height value to a continuous numeric value."""
    try:
        # Try to extract a numeric value from the string
        # Height may be in format like "height: 180cm" or similar
        if value is None:
            return None
        
        value = str(value).lower()
        # Look for height patterns
        if 'height' in value:
            # Extract numeric part - look for digits
            import re
            height_match = re.search(r'(\d+\.?\d*)', value)
            if height_match:
                return float(height_match.group(1))
        # If it's just a number, try to convert directly
        elif value.replace('.', '', 1).isdigit():
            return float(value)
        return None
    except Exception as e:
        print(f"Error converting trait: {e}")
        return None

def convert_age(value):
    """Convert age value to a continuous numeric value."""
    try:
        if value is None:
            return None
        
        value = str(value).lower()
        # Look for age patterns
        if 'age' in value or 'years' in value:
            # Extract numeric part
            import re
            age_match = re.search(r'(\d+\.?\d*)', value)
            if age_match:
                return float(age_match.group(1))
        # If it's just a number, try to convert directly
        elif value.replace('.', '', 1).isdigit():
            return float(value)
        return None
    except Exception as e:
        print(f"Error converting age: {e}")
        return None

def convert_gender(value):
    """Convert gender value to binary (0 for female, 1 for male)."""
    try:
        if value is None:
            return None
        
        value = str(value).lower()
        # Check for gender/sex indicators
        if 'female' in value or 'f' == value.strip() or 'f:


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# Based on reviewing the gene identifiers in the gene expression data, I can see they are numeric 
# identifiers (like 7, 8, 15, 18, etc.) rather than human gene symbols (which would be something 
# like BRCA1, TP53, etc.)
# 
# These appear to be probe IDs from a microarray platform, which need to be mapped to human gene symbols
# for proper biological interpretation. The numeric format is typical of Affymetrix or similar microarray 
# platforms where probes are identified by numbers rather than gene names.

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 7: Gene Identifier Mapping

In [ ]:
# Inspect the gene identifier columns in the annotation dataframe
print("Column names in gene annotation dataframe:")
print(gene_annotation.columns)

# Based on previous output, I can see that:
# - The gene expression data uses numeric IDs (like 7, 8, 15...)
# - The gene annotation dataframe has an 'ID' column that appears to contain similar numeric identifiers
# - The 'GENE_SYMBOL' column contains the human gene symbols we need

# 1. Identify the mapping columns
probe_id_col = 'ID'
gene_symbol_col = 'GENE_SYMBOL'

# 2. Get the gene mapping dataframe by extracting the two relevant columns
mapping_df = get_gene_mapping(gene_annotation, probe_id_col, gene_symbol_col)
print(f"Number of probes with gene mapping: {len(mapping_df)}")
print(f"Sample of the mapping dataframe:")
print(mapping_df.head())

# 3. Apply gene mapping to convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Dimensions of gene expression data after mapping: {gene_data.shape}")
print(f"Sample of gene symbols and expression values:")
print(gene_data.head())

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 8: Data Normalization and Linking

In [ ]:
import numpy as np
import os

# 1. Let's load the gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# Extract gene expression data
gene_data = get_genetic_data(matrix_file)

# Normalize gene symbols using the NCBI Gene database information
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to the output file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Link clinical and genetic data
# In Step 2, we determined that Height data is not available in this dataset (trait_row = None)
# Create a minimal clinical dataframe with the trait column (containing NaNs)
sample_ids = normalized_gene_data.columns
minimal_clinical_df = pd.DataFrame(index=sample_ids)
minimal_clinical_df[trait] = np.nan  # Add the trait column with NaN values

# If we have age and gender data from Step 2, add those columns
if age_row is not None:
    minimal_clinical_df['Age'] = get_feature_data(clinical_data, age_row, 'Age', convert_age).iloc[0]

if gender_row is not None:
    minimal_clinical_df['Gender'] = get_feature_data(clinical_data, gender_row, 'Gender', convert_gender).iloc[0]

minimal_clinical_df.index.name = 'Sample'

# Save this minimal clinical data for reference
os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
minimal_clinical_df.to_csv(out_clinical_data_file)
print(f"Clinical data saved to {out_clinical_data_file}")

# Create a linked dataset 
linked_data = pd.concat([minimal_clinical_df, normalized_gene_data.T], axis=1)
linked_data.index.name = 'Sample'
print(f"Linked data shape: {linked_data.shape}")

# We would normally handle missing values here, but since all trait values are missing,
# the dataset will be empty after removing samples with missing trait values
# Therefore, we'll skip that step

# 4 & 5. Validate and save cohort information
# Since trait_row was None in Step 2, we know Height data is not available
is_gene_available = len(normalized_gene_data) > 0
is_trait_available = False  # Height data is not available

note = "Dataset contains gene expression data but no Height measurements. This dataset is not usable for studying Height associations."

# For datasets without trait data, we set is_biased to False
# This indicates the dataset is not usable due to missing trait data, not due to bias
is_biased = False

# Final validation
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available, 
    is_biased=is_biased,
    df=linked_data,
    note=note
)

# 6. Since there is no trait data, the dataset is not usable for our association study
# So we should not save it to out_data_file
print(f"Dataset usability: {is_usable}")
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset does not contain Height data and cannot be used for association studies.")